In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# dataset & dataloader

In [ ]:
### Dataset Construction
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

batch_size = 64

### transform dataset to dataloader
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'shape of X [N,C,H,W]:{X.shape}')
    print(f'shape of y:{y.shape}{y.dtype}')
    break

shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64


# model

In [5]:
# device
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f'using {device} device')

### define the model
class classification_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = classification_nn().to(device)
print(model)

using cuda device
classification_nn(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# optimizer & training

In [ ]:
### loss function & optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)

In [10]:
### training and test
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    # load data in each for iteration
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # backward propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # log training process
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>8f}, [{current:>5d}/{size:>5d}]')

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, correct = 0, 0
    # no need to calculate gradient
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).sum().item()
        
        test_loss /= num_batches
        correct /= size
        
        print(f'Test accuracy: {correct * 100:>0.1f}%\nAvg loss: {test_loss:>8f}\n')

### train & evaluate for several epoches
epochs = 10
for epoch in range(epochs):
    print(f'epoch:{epoch}\n---------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

epoch:0
---------------------------------
loss: 0.750900, [   64/60000]
loss: 0.837612, [ 6464/60000]
loss: 0.602436, [12864/60000]
loss: 0.822067, [19264/60000]
loss: 0.738139, [25664/60000]
loss: 0.720804, [32064/60000]
loss: 0.801757, [38464/60000]
loss: 0.786390, [44864/60000]
loss: 0.781527, [51264/60000]
loss: 0.742884, [57664/60000]
Test accuracy: 73.0%
Avg loss: 0.743115

epoch:1
---------------------------------
loss: 0.717341, [   64/60000]
loss: 0.809637, [ 6464/60000]
loss: 0.574584, [12864/60000]
loss: 0.799627, [19264/60000]
loss: 0.718950, [25664/60000]
loss: 0.698396, [32064/60000]
loss: 0.778265, [38464/60000]
loss: 0.771614, [44864/60000]
loss: 0.761615, [51264/60000]
loss: 0.722102, [57664/60000]
Test accuracy: 74.1%
Avg loss: 0.722477

epoch:2
---------------------------------
loss: 0.687770, [   64/60000]
loss: 0.784272, [ 6464/60000]
loss: 0.550359, [12864/60000]
loss: 0.780306, [19264/60000]
loss: 0.701963, [25664/60000]
loss: 0.679348, [32064/60000]
loss: 0.7563

# save & load models

In [11]:
### save model parameters (but not structure)
save_path = 'model.pth'
torch.save(model.state_dict(), save_path)
print(f'save model to {save_path}')

### load model
model = classification_nn().to(device)
model.load_state_dict(torch.load(save_path, weights_only=True))

save model to model.pth


<All keys matched successfully>

# evaluate

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

with torch.no_grad():
    model.eval()
    x, y = test_data[0][0], test_data[0][1]
    x = x.to(device)
    pred = model(x)
    print(f'model prediction: {classes[pred[0].argmax(0)]}\tground truth: {classes[y]}')

model prediction: Ankle boot	ground truth: Ankle boot
